In [ ]:
#Recomendação de filmes por similaridade dos cossenos
#Ellen Ozores e Mel Yukari



In [ ]:
import numpy as np
import pandas as pd
import string
import nltk

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity



In [ ]:
filmes = pd.read_csv('Filmes.csv', sep=',')
dados = pd.read_csv('Dados.csv', sep=',')
tags = pd.read_csv('Tags.csv', sep=",")

In [ ]:
filmes.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


In [ ]:
dados.head()

,Name,Year,Stars,Score,Time,Votes,Total,Tags,Directors_Cast,Discription
0,Pulp Fiction,1994,8.9,94.0,154,"1,871,051",$107.93,\nCrime Drama,Quentin Tarantino John Travolta Uma Thurman Sa...,\nThe lives of two mob hitmen a boxer a gangst...
1,The Amazing Spider-Man 2,2014,6.6,53.0,142,"425,529",$202.85,\nAction Adventure Fantasy,Marc Webb Andrew Garfield Emma Stone Jamie Fox...,\nWhen New York is put under siege by Oscorp i...
2,The Shawshank Redemption,1994,9.3,80.0,142,"2,409,436",$28.34,\nDrama,Frank Darabont Tim Robbins Morgan Freeman Bob ...,\nTwo imprisoned men bond over a number of yea...
3,Star Wars: Episode IV - A New Hope,1977,8.6,90.0,121,"1,255,464",$322.74,\nAction Adventure Fantasy,George Lucas Mark Hamill Harrison Ford Carrie ...,\nLuke Skywalker joins forces with a Jedi Knig...
4,Back to the Future,1985,8.5,87.0,116,"1,087,878",$210.61,\nAdventure Comedy Sci-Fi,Robert Zemeckis Michael J. Fox Christopher Llo...,\nMarty McFly a 17-year-old high school studen...


In [ ]:
tags.head()

,movieId,tag
0,1,"Owned,imdb top 250,Pixar,Pixar,time travel,chi..."
1,2,"Robin Williams,time travel,fantasy,based on ch..."
2,3,"funny,best friend,duringcreditsstinger,fishing..."
3,4,"based on novel or book,chick flick,divorce,int..."
4,5,"aging,baby,confidence,contraception,daughter,g..."


In [ ]:
filmes['movieId'] = filmes['movieId'].apply(lambda x: str(x))

In [ ]:
df = filmes.merge(dados, left_on='title', right_on= 'Name', how='left')
df = df.merge(tags, left_on='movieId', right_on='movieId', how='left')
df['infos'] = df['genres'] + df['Directors_Cast'] + df['Discription'] + df['tag']
df.head()

,movieId,title,genres,year,Name,Year,Stars,Score,Time,Votes,Total,Tags,Directors_Cast,Discription,tag,infos
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995,Toy Story,1995,8.3,95.0,81,"908,794",$191.80,\nAnimation Adventure Comedy,John Lasseter Tom Hanks Tim Allen Don Rickles ...,\nA cowboy doll is profoundly threatened and j...,"Owned,imdb top 250,Pixar,Pixar,time travel,chi...",Adventure|Animation|Children|Comedy|FantasyJoh...
1,2,Jumanji,Adventure|Children|Fantasy,1995,Jumanji,1995,7.0,39.0,104,"316,485",$100.48,\nAdventure Comedy Family,Joe Johnston Robin Williams Kirsten Dunst Bonn...,\nWhen two kids find and play a magical board ...,"Robin Williams,time travel,fantasy,based on ch...",Adventure|Children|FantasyJoe Johnston Robin W...
2,3,Grumpier Old Men,Comedy|Romance,1995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"funny,best friend,duringcreditsstinger,fishing...",NaN
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995,Waiting to Exhale,1995,6.0,NaN,124,"9,911",$67.05,\nComedy Drama Romance,Forest Whitaker Whitney Houston Angela Bassett...,NaN,"based on novel or book,chick flick,divorce,int...",NaN
4,5,Father of the Bride Part II,Comedy,1995,Father of the Bride Part II,1995,6.1,49.0,106,"35,472",$76.59,\nComedy Family Romance,Charles Shyer Steve Martin Diane Keaton Martin...,\nGeorge Banks must deal not only with the pre...,"aging,baby,confidence,contraception,daughter,g...",ComedyCharles Shyer Steve Martin Diane Keaton ...


In [ ]:
df['infos']= df['infos'].str.lower()
df['title']= df['title'].str.replace(',', '', regex=False)
df.head()

,movieId,title,genres,year,Name,Year,Stars,Score,Time,Votes,Total,Tags,Directors_Cast,Discription,tag,infos
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995,Toy Story,1995,8.3,95.0,81,"908,794",$191.80,\nAnimation Adventure Comedy,John Lasseter Tom Hanks Tim Allen Don Rickles ...,\nA cowboy doll is profoundly threatened and j...,"Owned,imdb top 250,Pixar,Pixar,time travel,chi...",adventure|animation|children|comedy|fantasyjoh...
1,2,Jumanji,Adventure|Children|Fantasy,1995,Jumanji,1995,7.0,39.0,104,"316,485",$100.48,\nAdventure Comedy Family,Joe Johnston Robin Williams Kirsten Dunst Bonn...,\nWhen two kids find and play a magical board ...,"Robin Williams,time travel,fantasy,based on ch...",adventure|children|fantasyjoe johnston robin w...
2,3,Grumpier Old Men,Comedy|Romance,1995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"funny,best friend,duringcreditsstinger,fishing...",NaN
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995,Waiting to Exhale,1995,6.0,NaN,124,"9,911",$67.05,\nComedy Drama Romance,Forest Whitaker Whitney Houston Angela Bassett...,NaN,"based on novel or book,chick flick,divorce,int...",NaN
4,5,Father of the Bride Part II,Comedy,1995,Father of the Bride Part II,1995,6.1,49.0,106,"35,472",$76.59,\nComedy Family Romance,Charles Shyer Steve Martin Diane Keaton Martin...,\nGeorge Banks must deal not only with the pre...,"aging,baby,confidence,contraception,daughter,g...",comedycharles shyer steve martin diane keaton ...


In [ ]:
nltk.download('stopwords')
from nltk.corpus import stopwords
",".join(stopwords.words('english'))
stop_words= set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def remove_stop(x):
  return ",".join([word for word in str(x).split() if word not in stop_words])
df['infos']= df['infos'].apply(lambda x : remove_stop(x))

In [ ]:
df.head(10)

,movieId,title,genres,year,Name,Year,Stars,Score,Time,Votes,Total,Tags,Directors_Cast,Discription,tag,infos
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995,Toy Story,1995,8.3,95.0,81,"908,794",$191.80,\nAnimation Adventure Comedy,John Lasseter Tom Hanks Tim Allen Don Rickles ...,\nA cowboy doll is profoundly threatened and j...,"Owned,imdb top 250,Pixar,Pixar,time travel,chi...",adventure|animation|children|comedy|fantasyjoh...
1,2,Jumanji,Adventure|Children|Fantasy,1995,Jumanji,1995,7.0,39.0,104,"316,485",$100.48,\nAdventure Comedy Family,Joe Johnston Robin Williams Kirsten Dunst Bonn...,\nWhen two kids find and play a magical board ...,"Robin Williams,time travel,fantasy,based on ch...","adventure|children|fantasyjoe,johnston,robin,w..."
2,3,Grumpier Old Men,Comedy|Romance,1995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"funny,best friend,duringcreditsstinger,fishing...",nan
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995,Waiting to Exhale,1995,6.0,NaN,124,"9,911",$67.05,\nComedy Drama Romance,Forest Whitaker Whitney Houston Angela Bassett...,NaN,"based on novel or book,chick flick,divorce,int...",nan
4,5,Father of the Bride Part II,Comedy,1995,Father of the Bride Part II,1995,6.1,49.0,106,"35,472",$76.59,\nComedy Family Romance,Charles Shyer Steve Martin Diane Keaton Martin...,\nGeorge Banks must deal not only with the pre...,"aging,baby,confidence,contraception,daughter,g...","comedycharles,shyer,steve,martin,diane,keaton,..."
5,6,Heat,Action|Crime|Thriller,1995,Heat,1995,8.2,76.0,170,"593,063",$67.44,\nCrime Drama Thriller,Michael Mann Al Pacino Robert De Niro Val Kilm...,\nA group of professional bank robbers start t...,"imdb top 250,great acting,realistic action,sus...","action|crime|thrillermichael,mann,al,pacino,ro..."
6,6,Heat,Action|Crime|Thriller,1995,Heat,1986,5.6,NaN,101,"1,871",$2.79,\nAction Crime Drama,Dick Richards Jerry Jameson Burt Reynolds Kare...,NaN,"imdb top 250,great acting,realistic action,sus...",nan
7,7,Sabrina,Comedy|Romance,1995,Sabrina,1995,6.3,56.0,127,"37,724",$53.67,\nComedy Drama Romance,Sydney Pollack Harrison Ford Julia Ormond Greg...,\nAn ugly duckling having undergone a remarkab...,"remake,chauffeur,fusion,long island,millionair...","comedy|romancesydney,pollack,harrison,ford,jul..."
8,8,Tom and Huck,Adventure|Children,1995,Tom and Huck,1995,5.5,NaN,97,"10,095",$23.74,\nAdventure Comedy Drama,Peter Hewitt Jonathan Taylor Thomas Brad Renfr...,NaN,"based on a book,Mark Twain,Peter Hewitt,adapte...",nan
9,9,Sudden Death,Action,1995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"explosive,hostage,terrorist,vice president,Jea...",nan


In [ ]:
vectorizer = TfidfVectorizer()
tfidf = vectorizer.fit_transform(df['infos'].apply(lambda x: np.str_(x)))

In [ ]:
sim = cosine_similarity(tfidf)
sim

array([[1.        , 0.05379694, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.05379694, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 1.        , 1.        ,
        1.        ],
       ...,
       [0.        , 0.        , 1.        , ..., 1.        , 1.        ,
        1.        ],
       [0.        , 0.        , 1.        , ..., 1.        , 1.        ,
        1.        ],
       [0.        , 0.        , 1.        , ..., 1.        , 1.        ,
        1.        ]])

In [ ]:
sim_df = pd.DataFrame(sim, columns=df['title'], index=df['title']).drop_duplicates()
sim_df.head()

title,Toy Story,Jumanji,Grumpier Old Men,Waiting to Exhale,Father of the Bride Part II,Heat,Heat,Sabrina,Tom and Huck,Sudden Death,...,Gintama: The Movie,anohana: The Flower We Saw That Day - The Movie,Silver Spoon,Love Live! The School Idol Movie,Jon Stewart Has Left the Building,Black Butler: Book of the Atlantic,No Game No Life: Zero,Flint,Bungo Stray Dogs: Dead Apple,Andrew Dice Clay: Dice Rules
title,,,,,,,,,,,,,,,,,,,,,
Toy Story,1.000000,0.053797,0.0,0.0,0.015886,0.008608,0.0,0.003633,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Jumanji,0.053797,1.000000,0.0,0.0,0.068269,0.001259,0.0,0.022045,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Grumpier Old Men,0.000000,0.000000,1.0,1.0,0.000000,0.000000,1.0,0.009479,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
Father of the Bride Part II,0.015886,0.068269,0.0,0.0,1.000000,0.000827,0.0,0.008812,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Heat,0.008608,0.001259,0.0,0.0,0.000827,1.000000,0.0,0.014256,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#Os nomes dos filmes estão todos em inglês
nome= input("Qual filme você quer recomendação? Digite o título em inglês e com as letras iniciais em maiúsculo:"  )
if nome not in sim_df.columns:
  print("Esse filme não foi encontrado no banco de dados!")
final_df = pd.DataFrame(sim_df[nome].sort_values(ascending=False))
final_df.columns = ["Filmes que você gostaria de assistir: "]
final_df.head(10)


Qual filme você quer recomendação? Digite o título em inglês e com as letras iniciais em maiúsculo:Thor


,Filmes que você gostaria de assistir:
title,
Thor,1.000000
Thor: The Dark World,0.718366
Thor: Ragnarok,0.601772
Avengers: Age of Ultron,0.570866
Captain America: The First Avenger,0.560111
Ant-Man,0.521466
Elektra,0.493285
Fantastic Four,0.476977
Fantastic Four,0.468677
